In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk  
from itertools import repeat
from keras.models import Sequential
from keras.layers import Dense
import keras
from tensorflow.keras import backend
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [2]:
df = pd.DataFrame()
df  = pd.read_csv('Tittles.csv',encoding='utf-8',header=None)
Tittles = df[0]
df.head()

,0
0,'Ci: \tUhsienrks \iDfa nyioeul awFaen7t\ Daels...
1,'Cl:o\oUksse rlsi\kDea ntiheelraeF es7 \fDiexs...
2,'Cw:h\yU stehres \hDaatnei emlaanF e 7d\oDnees...
3,'Ct:h\aUts esr sa\ DmaanniaegleamFeen7t\ Diess...
4,'Cb:i\tUcshe rpsl\eDaasnei e l ayFoeu7 \aDrees...


In [3]:
#Loading Info
documents = []
partition = []
X_train = []
X_test = []
Y_train = []
Y_test = []
for tittle in Tittles:
    tittle = tittle.replace(" ", "")
    tittle = tittle.replace("'", "")
    sub1 = tittle.find("\pos\\")
    name = tittle[sub1+6:]
    sub2 = name.find(".txt.")
    name = name[:sub2+4]
    if "trrainninng" in name:
        partition.append(0)
        if "posi" in name:
            Y_train.append(1)
        else:
            Y_train.append(0)
    elif "teestiinng" in name:
        partition.append(1)
        if "posi" in name:
            Y_test.append(1)
        else:
            Y_test.append(0)
    #print(tittle)
    #print(name)
    f=open("Files/pos/"+name, "r")
    document = (f.read()) 
    #print(document)
    documents.append(str(document))

Y_train = np.asarray(Y_train)
Y_test = np.asarray(Y_test)
print("Documents of dimension: (",len(documents),")")
print("Partition of dimension: (",len(partition),")")
print("Y_train of dimension: (",Y_train.shape,")")
print("Y_test of dimension: (",Y_test.shape,")")

#import csv
#wtr = csv.writer(open ('out.csv', 'w'), delimiter=',', lineterminator='\n')
#for x in documents : wtr.writerow ([x])

Documents of dimension: ( 8798 )
Partition of dimension: ( 8798 )
Y_train of dimension: ( (6161,) )
Y_test of dimension: ( (2637,) )


In [4]:
df = pd.DataFrame()
df  = pd.read_csv('out.csv',encoding='utf-8',header=None)
documents = df[0]
print("Documents: ",documents[0])

Documents:  I think if you want all the trophies, you SHOULD have to play ALL the game  People work hard on that game & if you can t be bothered to play it all, then you shouldn t CARE so much about not getting a trophy for it 



In [5]:
df = pd.DataFrame()
df  = pd.read_csv('DiccII.csv',encoding='utf-8',header=None)
uniqueWords = df[0]
df.head()

,0
0,10th
1,1st
2,21st
3,2nd
4,3rd


In [6]:
#Documents Cleanning
documents = documents.str.replace('"',' ') 
documents = documents.str.replace('.',' ') 
documents = documents.str.replace("''",' ')
documents = documents.str.lower()
documents = documents.str.replace('[^\w\s]',' ') 
for i in range(len(documents )):
    # tokenize
    documents [i] = nltk.word_tokenize(documents [i])
    # remove blanck spaces
    documents [i] = [re.sub(r'\W',' ',word) for word in documents [i] ]
print("Ther are :",len(documents)," 0:", documents[0])

Ther are : 8798  0: ['i', 'think', 'if', 'you', 'want', 'all', 'the', 'trophies', 'you', 'should', 'have', 'to', 'play', 'all', 'the', 'game', 'people', 'work', 'hard', 'on', 'that', 'game', 'if', 'you', 'can', 't', 'be', 'bothered', 'to', 'play', 'it', 'all', 'then', 'you', 'shouldn', 't', 'care', 'so', 'much', 'about', 'not', 'getting', 'a', 'trophy', 'for', 'it']


In [7]:
# Result of TMG toolbox
uniqueWords = np.asarray(uniqueWords)
print(uniqueWords.shape)
print(uniqueWords[3525]) #Cuidado es -1

(4011,)
think


In [8]:
# Saving each unique word index
word_idx = {}
idx = 0
for token in uniqueWords:
    word_idx[token] =  idx
    idx += 1
print(word_idx['think'])

3525


In [9]:
sentence_vectors = []
for document in documents:
    sent_vec = []
    for token in uniqueWords:
        if token in document:
            tf = document.count(token)
            sent_vec.append(tf)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [10]:
sentence_vectors = np.asarray(sentence_vectors)
print("Matrix of dimension: (",sentence_vectors.shape[0],",",sentence_vectors.shape[1],")")

Matrix of dimension: ( 8798 , 4011 )


In [11]:
# Identifying the document with more terms, docs include terns not present in sentence_vectors
maxlen_doc = []
for doc in documents:
    maxlen_doc.append(len(doc))
maxlen_doc = max(maxlen_doc)
print("Maximum document longitu (may include not present terms): ",(maxlen_doc))  

Maximum document longitu (may include not present terms):  4066


In [12]:
num_docs = sentence_vectors.shape[0]
idx_matriz = np.zeros((num_docs, maxlen_doc)) # idx_matriz represents "strain+stest" in Matlab
row = 0
for doc in documents:
    # If tern not in dicc, add '0'
    #doc_inx = [ word_idx[x] if x in word_idx else 0 for x in doc] 
    # If tern not in dicc, not takenin account
    doc_inx = [word_idx[x] for x in doc if x in word_idx]
    doc_inx.extend(repeat(0, maxlen_doc - len(doc_inx)))  # Here we are padding, but it is NOT necessary
    idx_matriz[row] = (doc_inx)
    row += 1
    # Like this ingnore those words tht are not in dic : doc_inx = [wordfreq[x] for x in doc if x in wordfreq]
    #maximum.append(len(doc_inx))
print("Matrix of dimension: (",idx_matriz.shape[0],",",idx_matriz.shape[1],")")

Matrix of dimension: ( 8798 , 4066 )


In [13]:
#Divide data in Trainning and Testing sets
pos = 0
X_train,X_test,S_train,S_test = [],[],[],[]
for i in partition:
    if i == 0:
        X_train.append(sentence_vectors[pos])
        #Y_train.append(labels[pos])
        S_train.append(idx_matriz[pos])
    else:
        X_test.append(sentence_vectors[pos])
        #Y_test.append(labels[pos])
        S_test.append(idx_matriz[pos])
    pos += 1
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
#Y_train = np.asarray(Y_train)
#Y_test = np.asarray(Y_test)
S_train = np.asarray(S_train)
S_test = np.asarray(S_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)

(6161, 4011)
(6161,)
(2637, 4011)
(2637,)
(6161, 4066)
(2637, 4066)


In [14]:
def csa (X,Y,XT):
    num_docs = X.shape[0] # number of trainning docs
    num_terms = X.shape[1]
    num_class = 2
    TR = np.zeros((num_terms, num_class))
    DL = np.sum(X, axis=1) # sum each elemet row, wich is each df element of a document
    DL[DL == 0] = 1 #Those documents with not terms in dicc, are empty .
   
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X[docs,i] # tf_v is tern frecuency of each term
            TR[i][j] = sum(np.log2(1 + tf_v / DL [docs]))            
    print("PRB: (",TR.shape[0],",",TR.shape[1],")")

    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    n1[n1 == 0] = 1
    Tik = TR / n1

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        suma = sum(Tik[i])
        if suma == 0:
            suma = 1
        Tik[i] = Tik[i] / suma # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X[i] != 0)[0] # extrac idxs =! 0
        a = X[i][termos] / DL[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos])) # alpha * tik; alpha is the df of each term
    # Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"    return Tik
    
    # Document Test Representation
    DLT = np.sum(XT, axis=1)
    DLT[DLT == 0] = 1
    num_docs = XT.shape[0]
    DRT = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(XT[i] != 0)[0] # extrac idxs =! 0
        a = XT[i][termos] / DLT[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DRT[i] = sum(np.multiply(a,Tik[termos]))

    return DR,DRT

In [15]:
DR,DRT = csa (X_train,Y_train,X_test)
print("DR: (",DR.shape[0],",",DR.shape[1],")")

PRB: ( 4011 , 2 )
Tik: ( 4011 , 2 )
DR: ( 6161 , 2 )


In [16]:
for i in DR:
    print(i)

[0.58291687 0.41708313]
[0.68015184 0.31984816]
[0.51554795 0.48445205]
[0.65136726 0.34863274]
[0.38838939 0.61161061]
[0.44624304 0.55375696]
[0.4123383 0.5876617]
[0.43194269 0.56805731]
[0.48184486 0.51815514]
[0.54191543 0.45808457]
[0.56781005 0.43218995]
[0.40196466 0.59803534]
[0.60797766 0.39202234]
[0.65716988 0.34283012]
[0.4379604 0.5620396]
[0.74131021 0.25868979]
[0.56342935 0.43657065]
[0.61492137 0.38507863]
[0.53766195 0.46233805]
[0.57539593 0.42460407]
[0.24160865 0.75839135]
[0.3928632 0.6071368]
[0.74869348 0.25130652]
[0.60509774 0.39490226]
[0.75698915 0.24301085]
[0.56513158 0.43486842]
[0.38166218 0.61833782]
[0.49802983 0.50197017]
[0.41610021 0.58389979]
[0.55606897 0.44393103]
[0.62602934 0.37397066]
[0.56277624 0.43722376]
[0.63169375 0.36830625]
[0.69761021 0.30238979]
[0.61015462 0.38984538]
[0.51506943 0.48493057]
[0.67323304 0.32676696]
[0.52529374 0.47470626]
[0.51797535 0.48202465]
[0.41709811 0.58290189]
[0.57943954 0.42056046]
[0.67106521 0.32893479

[0.61033002 0.38966998]
[0.56050631 0.43949369]
[0.55531073 0.44468927]
[0.51873891 0.48126109]
[0.47321541 0.52678459]
[0.67415117 0.32584883]
[0.53312015 0.46687985]
[0.51180469 0.48819531]
[0.45815238 0.54184762]
[0.5886423 0.4113577]
[0.60981979 0.39018021]
[0.58432762 0.41567238]
[0.65870231 0.34129769]
[0.49560577 0.50439423]
[0.04601469 0.95398531]
[0.5263728 0.4736272]
[0.57739554 0.42260446]
[0.63660346 0.36339654]
[0.20675728 0.79324272]
[0.579974 0.420026]
[0.6029506 0.3970494]
[0.20675728 0.79324272]
[0.63360027 0.36639973]
[0.72032823 0.27967177]
[0.45568925 0.54431075]
[0.50227696 0.49772304]
[0.64411322 0.35588678]
[0.45102814 0.54897186]
[0.52571043 0.47428957]
[0.61204523 0.38795477]
[0.52113079 0.47886921]
[0.52740814 0.47259186]
[0.60015968 0.39984032]
[0.54866346 0.45133654]
[0.4684038 0.5315962]
[0.48595933 0.51404067]
[0.64700916 0.35299084]
[0.66706162 0.33293838]
[0.77265163 0.22734837]
[0.66663529 0.33336471]
[0.46643434 0.53356566]
[0.53978021 0.46021979]
[0.6

[0.49761322 0.50238678]
[0.68939599 0.31060401]
[0.51075211 0.48924789]
[0.70186335 0.29813665]
[0.42318141 0.57681859]
[0.51906111 0.48093889]
[0.23912471 0.76087529]
[0.61110874 0.38889126]
[0.379068 0.620932]
[0.58572547 0.41427453]
[0.58308015 0.41691985]
[0.54968549 0.45031451]
[0.60563796 0.39436204]
[0.47553231 0.52446769]
[0.42277359 0.57722641]
[0.44383987 0.55616013]
[0.77114515 0.22885485]
[0.59506385 0.40493615]
[0.63982553 0.36017447]
[0.61933269 0.38066731]
[0.72829658 0.27170342]
[0.58542055 0.41457945]
[0.56025605 0.43974395]
[0.56152222 0.43847778]
[0.28111234 0.71888766]
[0.53978022 0.46021978]
[0.62030428 0.37969572]
[0.61642347 0.38357653]
[0.3587271 0.6412729]
[0.57216801 0.42783199]
[0.5815509 0.4184491]
[0.61425415 0.38574585]
[0.53603259 0.46396741]
[0.60342971 0.39657029]
[0.56950593 0.43049407]
[0.26738998 0.73261002]
[0.73662945 0.26337055]
[0.67466317 0.32533683]
[0.64192194 0.35807806]
[0.58538081 0.41461919]
[0.57456597 0.42543403]
[0.54569381 0.45430619]


[0.61909661 0.38090339]
[0.54372846 0.45627154]
[0.45085416 0.54914584]
[0.57750936 0.42249064]
[0.6832211 0.3167789]
[0.68458118 0.31541882]
[0.42636726 0.57363274]
[0.69619464 0.30380536]
[0.47460786 0.52539214]
[0.55450841 0.44549159]
[0.42319458 0.57680542]
[0.4781491 0.5218509]
[0.68051295 0.31948705]
[0.55486118 0.44513882]
[0.4861982 0.5138018]
[0.52608009 0.47391991]
[0.48948908 0.51051092]
[0.40040462 0.59959538]
[0.42728881 0.57271119]
[0.50594251 0.49405749]
[0.61765851 0.38234149]
[0.51675214 0.48324786]
[0.45423793 0.54576207]
[0.64268047 0.35731953]
[0.6365161 0.3634839]
[0.65894892 0.34105108]
[0.5572789 0.4427211]
[0.56842918 0.43157082]
[0.62235283 0.37764717]
[0.51982125 0.48017875]
[0.49993212 0.50006788]
[0.58646474 0.41353526]
[0.45652872 0.54347128]
[0.702747 0.297253]
[0.4833813 0.5166187]
[0.52340159 0.47659841]
[0.55126519 0.44873481]
[0.55824217 0.44175783]
[0.65202969 0.34797031]
[0.57539396 0.42460604]
[0.33625559 0.66374441]
[0.57490254 0.42509746]
[0.43984

[0.82758196 0.17241804]
[0.49778112 0.50221888]
[0.64706425 0.35293575]
[0.60083381 0.39916619]
[0.59310183 0.40689817]
[0.64135901 0.35864099]
[0.31123719 0.68876281]
[0.53841216 0.46158784]
[0.51294506 0.48705494]
[0.63070256 0.36929744]
[0.45099687 0.54900313]
[0.32393624 0.67606376]
[0.74488353 0.25511647]
[0.49321355 0.50678645]
[0.53522559 0.46477441]
[0.47424943 0.52575057]
[0.46277599 0.53722401]
[0.68057472 0.31942528]
[0.62754091 0.37245909]
[0.57420006 0.42579994]
[0.61184368 0.38815632]
[0.65266101 0.34733899]
[0.62458873 0.37541127]
[0.30291227 0.69708773]
[0.48257749 0.51742251]
[0.57130346 0.42869654]
[0.42748603 0.57251397]
[0.3721582 0.6278418]
[0.57484225 0.42515775]
[0.51522435 0.48477565]
[0.57579085 0.42420915]
[1. 0.]
[0.4662784 0.5337216]
[0.53760974 0.46239026]
[0.53111558 0.46888442]
[0.54024793 0.45975207]
[0.50953524 0.49046476]
[0.54483924 0.45516076]
[0.69695922 0.30304078]
[0.51818165 0.48181835]
[0.6835255 0.3164745]
[0.55494043 0.44505957]
[0.68785094 0.

...

[0.44304299 0.55695701]
[0.55348365 0.44651635]
[0.43736599 0.56263401]
[0.46522588 0.53477412]
[0.37691295 0.62308705]
[0.53917859 0.46082141]
[0.56721669 0.43278331]
[0.48809016 0.51190984]
[0.44185332 0.55814668]
[0.44784865 0.55215135]
[0.52421757 0.47578243]
[0.44460149 0.55539851]
[0.39047047 0.60952953]
[0.54911293 0.45088707]
[0.34241652 0.65758348]
[0.62376561 0.37623439]
[0.56665369 0.43334631]
[0.57751729 0.42248271]
[0.34175019 0.65824981]
[0.48035884 0.51964116]
[0.51290288 0.48709712]
[0.54651994 0.45348006]
[0.22125302 0.77874698]
[0.31769911 0.68230089]
[0.34559429 0.65440571]
[0.45439745 0.54560255]
[0.49501696 0.50498304]
[0.51075858 0.48924142]
[0.32045758 0.67954242]
[0.53296061 0.46703939]
[0.50904717 0.49095283]
[0.16022013 0.83977987]
[0.60138324 0.39861676]
[0.44239284 0.55760716]
[0.41164772 0.58835228]
[0.41297975 0.58702025]
[0.46883239 0.53116761]
[0.69435311 0.30564689]
[0.47048204 0.52951796]
[0.50988143 0.49011857]
[0.38786132 0.61213868]
[0.59520283 0.40

[0.48629466 0.51370534]
[0.35505796 0.64494204]
[0.35125587 0.64874413]
[0.57907101 0.42092899]
[0.64878144 0.35121856]
[0.43185054 0.56814946]
[0.52135996 0.47864004]
[0.57949037 0.42050963]
[0.66511954 0.33488046]
[0.64520587 0.35479413]
[0.49599979 0.50400021]
[0.53487124 0.46512876]
[0.53609042 0.46390958]
[0.58558032 0.41441968]
[0.54314572 0.45685428]
[0.47207102 0.52792898]
[0.36606302 0.63393698]
[0.4604568 0.5395432]
[0.47685954 0.52314046]
[0.40108733 0.59891267]
[0.47577248 0.52422752]
[0.50352091 0.49647909]
[0.57383465 0.42616535]
[0.45037959 0.54962041]
[0.33884102 0.66115898]
[0.05861662 0.94138338]
[0.61596934 0.38403066]
[0.48633794 0.51366206]
[0.60117256 0.39882744]
[0.49460865 0.50539135]
[0.59820004 0.40179996]
[0.45316159 0.54683841]
[0.55423952 0.44576048]
[0.39918284 0.60081716]
[0.5030363 0.4969637]
[0.88059607 0.11940393]
[0.50641983 0.49358017]
[0.41282417 0.58717583]
[0.52097748 0.47902252]
[0.87581888 0.12418112]
[0.05794235 0.94205765]
[0.54008682 0.459913

[0.93306724 0.06693276]
[0.50481641 0.49518359]
[0.56593619 0.43406381]
[0.51797837 0.48202163]
[0.50004949 0.49995051]
[0.31025418 0.68974582]
[0.3858651 0.6141349]
[0.52362744 0.47637256]
[0.35392835 0.64607165]
[0.40266367 0.59733633]
[0.48453055 0.51546945]
[0.45545064 0.54454936]
[0.52239092 0.47760908]
[0.62091496 0.37908504]
[0.53496652 0.46503348]
[0.50537423 0.49462577]
[0.61468516 0.38531484]
[0.69871376 0.30128624]
[0.55072786 0.44927214]
[0.57788089 0.42211911]
[0.45479562 0.54520438]
[0.40997806 0.59002194]
[0.33914903 0.66085097]
[0.55610805 0.44389195]
[0.44122472 0.55877528]
[0.5066876 0.4933124]
[0.38406183 0.61593817]
[0.6447831 0.3552169]
[0.58829934 0.41170066]
[0.326432 0.673568]
[0.58489963 0.41510037]
[0.47501104 0.52498896]
[0.51979899 0.48020101]
[0.31702507 0.68297493]
[0.44232194 0.55767806]
[0.45187221 0.54812779]
[0.49959321 0.50040679]
[0.57750583 0.42249417]
[0.6437345 0.3562655]
[0.26318154 0.73681846]
[0.20308291 0.79691709]
[0.57880971 0.42119029]
[0.5

[0.26001898 0.73998102]
[0.46096738 0.53903262]
[0.57117784 0.42882216]
[0.51012363 0.48987637]
[0.59956553 0.40043447]
[0.52352483 0.47647517]
[0.43334484 0.56665516]
[0.75692925 0.24307075]
[0.46549631 0.53450369]
[0.89038677 0.10961323]
[0.37600529 0.62399471]
[0.57593294 0.42406706]
[0.64043319 0.35956681]
[0.61807308 0.38192692]
[0.54106785 0.45893215]
[0.5885682 0.4114318]
[0.4800294 0.5199706]
[0.60403587 0.39596413]
[0.35857328 0.64142672]
[0.48670269 0.51329731]
[0.60922937 0.39077063]
[0.59880759 0.40119241]
[0.53042022 0.46957978]
[0.64801743 0.35198257]
[0.43750342 0.56249658]
[0.500795 0.499205]
[0.31025418 0.68974582]
[0.97883072 0.02116928]
[0.56510415 0.43489585]
[0.48731066 0.51268934]
[0.62935406 0.37064594]
[0.56270188 0.43729812]
[0.69285463 0.30714537]
[0.44526372 0.55473628]
[0.48410438 0.51589562]
[0.41753129 0.58246871]
[0.34817034 0.65182966]
[0.61198266 0.38801734]
[0.6587438 0.3412562]
[0.75883564 0.24116436]
[0.73180787 0.26819213]
[0.49490894 0.50509106]
[0

[0.5854038 0.4145962]
[0.34106004 0.65893996]


In [22]:
# Create NN model
num_class = 2
model = Sequential()
# Add model layers
#model.add(Dense(10, activation='sigmoid', input_shape=(num_class,)))


model.add(Dense(10, input_dim=num_class,
                kernel_regularizer=regularizers.l2(1e-14),
                activity_regularizer=regularizers.l1(1e-14)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
#opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [20]:
model.fit(DR, Y_train, epochs=100, batch_size=10, validation_data=(DRT, Y_test))


Train on 6161 samples, validate on 2637 samples
Epoch 1/100
6161/6161 [==============================] - 2s 296us/step - loss: 0.6446 - acc: 0.6554 - val_loss: 0.5765 - val_acc: 0.7372
Epoch 2/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.5962 - acc: 0.6583 - val_loss: 0.5420 - val_acc: 0.7448
Epoch 3/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.5476 - acc: 0.6931 - val_loss: 0.5041 - val_acc: 0.7615
Epoch 4/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.4983 - acc: 0.7510 - val_loss: 0.4729 - val_acc: 0.7827
Epoch 5/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.4529 - acc: 0.7979 - val_loss: 0.4474 - val_acc: 0.7956
Epoch 6/100
6161/6161 [==============================] - 1s 153us/step - loss: 0.4160 - acc: 0.8226 - val_loss: 0.4329 - val_acc: 0.8074
Epoch 7/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.3887 - acc: 0.8357 - val_loss: 0.4230 - val_acc:

6161/6161 [==============================] - 1s 152us/step - loss: 0.3233 - acc: 0.8523 - val_loss: 0.4295 - val_acc: 0.8187
Epoch 59/100
6161/6161 [==============================] - 1s 150us/step - loss: 0.3233 - acc: 0.8538 - val_loss: 0.4313 - val_acc: 0.8187
Epoch 60/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.3230 - acc: 0.8533 - val_loss: 0.4301 - val_acc: 0.8138
Epoch 61/100
6161/6161 [==============================] - 1s 152us/step - loss: 0.3229 - acc: 0.8539 - val_loss: 0.4305 - val_acc: 0.8138
Epoch 62/100
6161/6161 [==============================] - 1s 149us/step - loss: 0.3235 - acc: 0.8531 - val_loss: 0.4304 - val_acc: 0.8180
Epoch 63/100
6161/6161 [==============================] - 1s 174us/step - loss: 0.3226 - acc: 0.8547 - val_loss: 0.4296 - val_acc: 0.8161
Epoch 64/100
6161/6161 [==============================] - 1s 154us/step - loss: 0.3235 - acc: 0.8542 - val_loss: 0.4295 - val_acc: 0.8187
Epoch 65/100
6161/6161 [=======================

In [21]:
del model

In [ ]:
retains = [0.01,0.06,0.11,0.16,0.21,0.26,0.31,0.36,0.41,0.46,0.51,0.56,0.61,0.66,0.71,0.76,0.81,0.86,0.91,1]
#retains = [0.00]
#retains = [0.110000000000000,0.160000000000000,0.210000000000000,0.260000000000000,0.310000000000000,0.360000000000000,0.410000000000000,0.460000000000000,0.510000000000000,0.560000000000000,0.610000000000000,0.660000000000000,0.710000000000000,0.760000000000000,0.810000000000000,0.860000000000000,0.910000000000000,1]

for j in range (0,len(retains)):
    print("---------------------------------------- ",retains[j]," ---------------------------------------------")
    rXtest = np.zeros((X_test.shape[0], X_test.shape[1]))
    for i in range(0,X_test.shape[0]): # iterating each test document
        noz = np.where(S_test[i] != 0)[0] # from our idx (ONLY INDXS) document matriz, extract idx != 0 (this step could be avoid if we do not agregate those 0)
        #ntermssf = round(len(noz)*retains[j]) # We obtain the number of terns with x% of longitud
        aa = S_test[i][noz]
        aa = np.unique(aa)
        aa = np.sort(aa)
        ntermssf = round(len(aa)*retains[j]) #This was the right thing, analize
        myox = 1 # minnum Termos = 1
        wdix = 0 # 
        freqtsof = np.zeros(X_test.shape[1])  
        # This vector activate the terms present in the Document until that longitud
        while myox <= ntermssf:
            if S_test[i,wdix] != 0:
                #freqtsof[int(S_test[i,wdix])-1] += 1 #If the same word appears more than once , just add one to the position
                freqtsof[int(S_test[i,wdix])] += 1
                myox += 1
            wdix += 1
        rXtest[i] = freqtsof
        
    DRS,DRTS = csa(X_train,Y_train,rXtest)
    

    #for i in DRTS:
      #  print(i)
    #print("............")
    #print(DRS[555])
    #print(DRTS[555])
    #DRS = np.nan_to_num(DRS)
    #DR_TS = np.nan_to_num(DR_TS)
    #print(DRS.shape)
    #print(DR_TS.shape)
    model.fit(DRS, Y_train, epochs=150, batch_size=10, validation_data=(DRTS, Y_test))
    #Xtestv = rXtest

----------------------------------------  0.01  ---------------------------------------------
PRB: ( 4011 , 2 )
Tik: ( 4011 , 2 )
Train on 6161 samples, validate on 2637 samples
Epoch 1/150
6161/6161 [==============================] - 1s 160us/step - loss: 0.3224 - acc: 0.8546 - val_loss: 1.1654 - val_acc: 0.3011
Epoch 2/150
6161/6161 [==============================] - 1s 168us/step - loss: 0.3231 - acc: 0.8531 - val_loss: 1.1779 - val_acc: 0.3011
Epoch 3/150
6161/6161 [==============================] - 1s 162us/step - loss: 0.3225 - acc: 0.8546 - val_loss: 1.1442 - val_acc: 0.3019
Epoch 4/150
6161/6161 [==============================] - 1s 162us/step - loss: 0.3229 - acc: 0.8542 - val_loss: 1.2220 - val_acc: 0.3011
Epoch 5/150
6161/6161 [==============================] - 1s 161us/step - loss: 0.3224 - acc: 0.8546 - val_loss: 1.1781 - val_acc: 0.3011
Epoch 6/150
6161/6161 [==============================] - 1s 160us/step - loss: 0.3225 - acc: 0.8567 - val_loss: 1.1982 - val_acc: 0.3011


6161/6161 [==============================] - 1s 150us/step - loss: 0.3228 - acc: 0.8539 - val_loss: 1.2070 - val_acc: 0.3011
Epoch 60/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3230 - acc: 0.8541 - val_loss: 1.1470 - val_acc: 0.3019
Epoch 61/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3230 - acc: 0.8529 - val_loss: 1.1907 - val_acc: 0.3011
Epoch 62/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3231 - acc: 0.8531 - val_loss: 1.1725 - val_acc: 0.3011
Epoch 63/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3230 - acc: 0.8549 - val_loss: 1.1377 - val_acc: 0.3019
Epoch 64/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3228 - acc: 0.8529 - val_loss: 1.2182 - val_acc: 0.3011
Epoch 65/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3227 - acc: 0.8539 - val_loss: 1.2432 - val_acc: 0.3007
Epoch 66/150
6161/6161 [=======================

6161/6161 [==============================] - 1s 150us/step - loss: 0.3227 - acc: 0.8520 - val_loss: 1.1975 - val_acc: 0.3011
Epoch 119/150
6161/6161 [==============================] - 1s 152us/step - loss: 0.3228 - acc: 0.8564 - val_loss: 1.1925 - val_acc: 0.3011
Epoch 120/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3225 - acc: 0.8564 - val_loss: 1.1076 - val_acc: 0.3022
Epoch 121/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3229 - acc: 0.8547 - val_loss: 1.1355 - val_acc: 0.3019
Epoch 122/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3230 - acc: 0.8534 - val_loss: 1.2696 - val_acc: 0.3007
Epoch 123/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3229 - acc: 0.8544 - val_loss: 1.2338 - val_acc: 0.3011
Epoch 124/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3230 - acc: 0.8542 - val_loss: 1.1632 - val_acc: 0.3011
Epoch 125/150
6161/6161 [================

6161/6161 [==============================] - 1s 171us/step - loss: 0.3231 - acc: 0.8546 - val_loss: 1.1112 - val_acc: 0.5529
Epoch 27/150
6161/6161 [==============================] - 1s 170us/step - loss: 0.3226 - acc: 0.8544 - val_loss: 1.0791 - val_acc: 0.5605
Epoch 28/150
6161/6161 [==============================] - 1s 188us/step - loss: 0.3230 - acc: 0.8536 - val_loss: 1.0996 - val_acc: 0.5575
Epoch 29/150
6161/6161 [==============================] - 1s 156us/step - loss: 0.3228 - acc: 0.8538 - val_loss: 1.1327 - val_acc: 0.5518
Epoch 30/150
6161/6161 [==============================] - 1s 152us/step - loss: 0.3228 - acc: 0.8547 - val_loss: 1.0875 - val_acc: 0.5605
Epoch 31/150
6161/6161 [==============================] - 1s 151us/step - loss: 0.3223 - acc: 0.8544 - val_loss: 1.0642 - val_acc: 0.5639
Epoch 32/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3225 - acc: 0.8547 - val_loss: 1.1560 - val_acc: 0.5495
Epoch 33/150
6161/6161 [=======================

6161/6161 [==============================] - 1s 150us/step - loss: 0.3228 - acc: 0.8538 - val_loss: 1.1212 - val_acc: 0.5518
Epoch 86/150
6161/6161 [==============================] - 1s 154us/step - loss: 0.3229 - acc: 0.8546 - val_loss: 1.0733 - val_acc: 0.5624
Epoch 87/150
6161/6161 [==============================] - 1s 148us/step - loss: 0.3229 - acc: 0.8546 - val_loss: 1.1217 - val_acc: 0.5518
Epoch 88/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3228 - acc: 0.8542 - val_loss: 1.1496 - val_acc: 0.5499
Epoch 89/150
6161/6161 [==============================] - 1s 150us/step - loss: 0.3228 - acc: 0.8562 - val_loss: 1.0849 - val_acc: 0.5605
Epoch 90/150
6161/6161 [==============================] - 1s 158us/step - loss: 0.3227 - acc: 0.8541 - val_loss: 1.1393 - val_acc: 0.5510
Epoch 91/150
6161/6161 [==============================] - 1s 152us/step - loss: 0.3229 - acc: 0.8541 - val_loss: 1.1249 - val_acc: 0.5518
Epoch 92/150
6161/6161 [=======================

...

6161/6161 [==============================] - 1s 155us/step - loss: 0.3226 - acc: 0.8542 - val_loss: 0.7161 - val_acc: 0.7436
Epoch 39/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3227 - acc: 0.8539 - val_loss: 0.7190 - val_acc: 0.7436
Epoch 40/150
6161/6161 [==============================] - 1s 152us/step - loss: 0.3227 - acc: 0.8539 - val_loss: 0.7133 - val_acc: 0.7497
Epoch 41/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3223 - acc: 0.8549 - val_loss: 0.7187 - val_acc: 0.7433
Epoch 42/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3227 - acc: 0.8541 - val_loss: 0.7173 - val_acc: 0.7433
Epoch 43/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3227 - acc: 0.8547 - val_loss: 0.7156 - val_acc: 0.7474
Epoch 44/150
6161/6161 [==============================] - 1s 152us/step - loss: 0.3227 - acc: 0.8552 - val_loss: 0.7186 - val_acc: 0.7440
Epoch 45/150
6161/6161 [=======================

6161/6161 [==============================] - 1s 156us/step - loss: 0.3228 - acc: 0.8538 - val_loss: 0.7148 - val_acc: 0.7490
Epoch 98/150
6161/6161 [==============================] - 1s 160us/step - loss: 0.3228 - acc: 0.8531 - val_loss: 0.7219 - val_acc: 0.7418
Epoch 99/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3228 - acc: 0.8551 - val_loss: 0.7231 - val_acc: 0.7414
Epoch 100/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3228 - acc: 0.8546 - val_loss: 0.7245 - val_acc: 0.7414
Epoch 101/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3228 - acc: 0.8544 - val_loss: 0.7160 - val_acc: 0.7474
Epoch 102/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3226 - acc: 0.8555 - val_loss: 0.7123 - val_acc: 0.7478
Epoch 103/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3227 - acc: 0.8557 - val_loss: 0.7206 - val_acc: 0.7429
Epoch 104/150
6161/6161 [==================

Epoch 5/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3226 - acc: 0.8549 - val_loss: 0.6784 - val_acc: 0.7490
Epoch 6/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3226 - acc: 0.8546 - val_loss: 0.6829 - val_acc: 0.7463
Epoch 7/150
6161/6161 [==============================] - 1s 163us/step - loss: 0.3229 - acc: 0.8533 - val_loss: 0.6830 - val_acc: 0.7463
Epoch 8/150
6161/6161 [==============================] - 1s 158us/step - loss: 0.3228 - acc: 0.8536 - val_loss: 0.6839 - val_acc: 0.7452
Epoch 9/150
6161/6161 [==============================] - 1s 158us/step - loss: 0.3225 - acc: 0.8539 - val_loss: 0.6760 - val_acc: 0.7520
Epoch 10/150
6161/6161 [==============================] - 1s 158us/step - loss: 0.3223 - acc: 0.8544 - val_loss: 0.6860 - val_acc: 0.7433
Epoch 11/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3225 - acc: 0.8549 - val_loss: 0.6788 - val_acc: 0.7459
Epoch 12/150
6161/6161 [===============

Epoch 64/150
6161/6161 [==============================] - 1s 154us/step - loss: 0.3228 - acc: 0.8547 - val_loss: 0.6790 - val_acc: 0.7471
Epoch 65/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3225 - acc: 0.8547 - val_loss: 0.6802 - val_acc: 0.7459
Epoch 66/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3225 - acc: 0.8526 - val_loss: 0.6825 - val_acc: 0.7463
Epoch 67/150
6161/6161 [==============================] - 1s 161us/step - loss: 0.3227 - acc: 0.8539 - val_loss: 0.6770 - val_acc: 0.7516
Epoch 68/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3229 - acc: 0.8531 - val_loss: 0.6800 - val_acc: 0.7467
Epoch 69/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3226 - acc: 0.8539 - val_loss: 0.6854 - val_acc: 0.7433
Epoch 70/150
6161/6161 [==============================] - 1s 152us/step - loss: 0.3228 - acc: 0.8546 - val_loss: 0.6825 - val_acc: 0.7463
Epoch 71/150
6161/6161 [==========

6161/6161 [==============================] - 1s 153us/step - loss: 0.3229 - acc: 0.8546 - val_loss: 0.6824 - val_acc: 0.7463
Epoch 124/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3228 - acc: 0.8544 - val_loss: 0.6782 - val_acc: 0.7493
Epoch 125/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3227 - acc: 0.8523 - val_loss: 0.6895 - val_acc: 0.7421
Epoch 126/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3221 - acc: 0.8539 - val_loss: 0.6864 - val_acc: 0.7433
Epoch 127/150
6161/6161 [==============================] - 1s 153us/step - loss: 0.3227 - acc: 0.8547 - val_loss: 0.6777 - val_acc: 0.7516
Epoch 128/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3227 - acc: 0.8538 - val_loss: 0.6820 - val_acc: 0.7459
Epoch 129/150
6161/6161 [==============================] - 1s 155us/step - loss: 0.3226 - acc: 0.8542 - val_loss: 0.6769 - val_acc: 0.7524
Epoch 130/150
6161/6161 [================